In [101]:
import cv2
import numpy as np

In [102]:
import torch

In [103]:
def try_func(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print(e)
        return None

In [104]:
import cmapy
cmap = cmapy.cmap('nipy_spectral')

In [105]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/handmotion/epoch5.pt')
model.to(device)

c:\projects\dig\venv\Lib\site-packages\torch\hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\ohyeo/.cache\torch\hub\master.zip
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
     ---------------------------------------- 1.1/1.1 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setu

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [106]:
colors = [
    (0, 0, 255),
    (0, 255, 0),
    (255, 0, 0),
]

In [109]:
cap = cv2.VideoCapture(0)
def test():
    fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기
    delay = int(1000/fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    history = np.zeros((height, width), dtype=np.int16)

    # 배경 제거 객체
    fgbg = cv2.createBackgroundSubtractorMOG2()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # 배경 제거 마스크 계산
        fgmask = fgbg.apply(frame)

        # make 24 frame of history based on fgmask
        pos = 15
        history = np.where(fgmask == 255, 255 + pos, history)
        history = np.where(history > -100, history - pos, history)

        # make clip for safety
        history = np.clip(history, 0, 255)
        history_frame = history.astype(np.uint8)

        # color map
        color_map_history = cv2.applyColorMap(history_frame, cmap)

        # detect
        results = model(color_map_history, size=640)

        result = frame.copy()
        for i in range(len(results.xyxy[0])):
            r = results.pandas().xyxy[0].iloc[i, :].values.tolist()
            
            x1, y1, x2, y2, confidence, cls_id, cls_name = r
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            c = colors[cls_id]
            name = f"{cls_id}. {cls_name} : {confidence:.2f}"

            cv2.rectangle(result, (x1, y1), (x2, y2), c, 2)
            cv2.putText(result, name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2)


        cv2.imshow('frame',frame)
        cv2.imshow('bgsub',fgmask)
        cv2.imshow('history', history_frame)
        cv2.imshow('cmaped', color_map_history)
        cv2.imshow('mhi_box', np.squeeze(results.render()))
        cv2.imshow('result', result)

        if cv2.waitKey(1) & 0xff == 27:
            break
try:
    test()
except Exception as e:
    raise e
finally:
    try_func(cap.release)
    try_func(cv2.destroyAllWindows)


print("Goodbye User")

Goodbye User


In [108]:
try_func(cap.release)
try_func(cv2.destroyAllWindows)